In [1]:
!pip install -U sentence-transformers==2.2.2 transformers==4.30.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.0 MB/s eta 0:00:0000:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=0251712e20164b4723e7290ec72fabf29764c6a0051204caffe9ed88e351ed06
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:


In [2]:
!pip install -q gradio==3.50.2 

In [2]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.6.1

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 98.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [5]:

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re
import string as s

from nltk.corpus import stopwords
from sklearn.metrics import make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix, roc_curve, auc

from nltk.corpus import stopwords
from wordcloud import WordCloud

from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.metrics  import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# import gradio as gr
# from sentence_transformers import SentenceTransformer
import joblib
import requests
import re
import html
from bs4 import BeautifulSoup

import numpy as np
import plotly.graph_objects as go

In [13]:
test_data = pd.read_json("/kaggle/input/bbc-news/bbc_news.json", lines=True)
test_data = test_data[test_data["label_text"] != "entertainment"]
label_dict = {
    "politics": 1,
    "sport": 2,
    "business": 3,
    "tech": 4
}
def mapper_dict(label):
    return label_dict[label]
    
test_data["ID"] = test_data["label_text"].map(mapper_dict)
test_data

,text,label,label_text,ID
1,sydney to host north v south game sydney will ...,2,sport,2
2,uk coal plunges into deeper loss shares in uk ...,1,business,3
3,blair joins school sailing trip the prime mini...,4,politics,1
4,bath faced with tindall ultimatum mike tindall...,2,sport,2
5,banker loses sexism claim a former executive a...,1,business,3
...,...,...,...,...
995,mobile multimedia slow to catch on there is no...,0,tech,4
996,owen determined to stay in madrid england forw...,2,sport,2
997,mobile tv tipped as one to watch scandinavians...,0,tech,4
998,stormy year for property insurers a string of ...,1,business,3


In [14]:
test_x = test_data.text
test_y = test_data.ID

In [15]:
import html
import re

def clean_html_noise(text: str) -> str:
   
    text = html.unescape(text)
    
   
    text = re.sub(r"<.*?>", "", text)
    
  
    text = re.sub(r"\s+", " ", text).strip()
    
    return text


test_x = test_x.apply(clean_html_noise)

In [16]:
def url_remove(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

test_x = test_x.apply(url_remove)


In [17]:

def word_tokenize(text):
    token = re.findall("[\w']+", text)
    return token

test_x = test_x.apply(word_tokenize)

test_x = test_x.apply(lambda x: [word.lower() for word in x])



In [18]:
def remove_stopwords(lst):
    stop_words = set(stopwords.words('english'))
    return [word for word in lst if word.lower() not in stop_words]

test_x = test_x.apply(remove_stopwords)


In [19]:
import string
def remove_punctuations(lst):
    return [''.join(char for char in word if char not in string.punctuation) for word in lst]

test_x = test_x.apply(remove_punctuations)



In [20]:
def remove_number(lst):
    return [''.join(char for char in word if char not in string.digits) for word in lst if word]

test_x = test_x.apply(remove_number)

In [21]:
def remove_extra_words(word_list):
    stopwords = ['href', 'lt', 'gt', 'ii', 'iii', 'ie', 'quot', 'com']
    cleaned_list = []
    for word in word_list:
        if word not in stopwords:
            cleaned_list.append(word)
    return cleaned_list
test_x = test_x.apply(remove_extra_words)


In [22]:

def join_words_with_space(sentence):
    joined_sentence = ' '.join(sentence)
    return joined_sentence

test_x = test_x.apply(join_words_with_space)

In [23]:
TFIDF_PIPELINE_PATH = "/kaggle/input/knn-tf-idf-complete/other/default/1/complete_pipeline.joblib"
tfidf_bundle = joblib.load(TFIDF_PIPELINE_PATH)
tfidf_vectorizer = tfidf_bundle["tfidf"]
chi2_selector    = tfidf_bundle["selector"]
knn_tfidf        = tfidf_bundle["knn"]

In [24]:
def classify(text):
    try:
        X = tfidf_vectorizer.transform([text])
        X = chi2_selector.transform(X)
        pred = knn_tfidf.predict(X)[0]
        return pred
    except Exception as e:
        return f"Error: {str(e)}"

In [28]:
pred = []
for _, row in test_data.iterrows():
    pred.append(classify(row["text"]))
test_data["prediction"] = pred

In [29]:

def eval_model(y_test,y_pred):
    results = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average='macro'),
        "recall": recall_score(y_test, y_pred, average='macro'),
        "f1": f1_score(y_test, y_pred, average='macro')
    }

    print("Test set accuracy: {:.4f}".format(results["accuracy"]))
    print("Test set precision: {:.4f}".format(results["precision"]))
    print("Test set recall: {:.4f}".format(results["recall"]))
    print("Test set F1: {:.4f}".format(results["f1"]))
    print("F1 per class:", f1_score(y_test, y_pred, average=None))
    return results


In [30]:
eval_model(test_data["ID"].tolist(), pred)

Test set accuracy: 0.7233
Test set precision: 0.7256
Test set recall: 0.7082
Test set F1: 0.7080
F1 per class: [0.59259259 0.7985348  0.72261072 0.71808511]


{'accuracy': 0.7233009708737864,
 'precision': 0.7256399941317246,
 'recall': 0.7082089891041162,
 'f1': 0.7079558050302731}